In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [24]:
df_cartoes = pd.read_csv("../data/raw/campeonato-brasileiro-cartoes.csv");
df_stats_full = pd.read_csv("../data/raw/campeonato-brasileiro-estatisticas-full.csv");
df_full = pd.read_csv("../data/raw/campeonato-brasileiro-full.csv");
df_gols = pd.read_csv("../data/raw/campeonato-brasileiro-gols.csv");

### Imprimindo as informações básicas de cada dataframe
print("Dataframe de cartões info")
print(df_cartoes.info())
print("Dataframe de stats info")
print(df_stats_full.info())
print("Dataframe todas as informações info")
print(df_full.info())
print("Dataframe de gols info")
print(df_gols.info())

### Imprimindo a "cabeça" de cada dataframe
print("Dataframe de cartoes")
print(df_cartoes.head())
print("Dataframe de stats")
print(df_stats_full.head())
print("Dataframe de informações")
print(df_full.head())
print("Dataframe de gols")
print(df_gols.head())

Dataframe de cartões info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18857 entries, 0 to 18856
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   partida_id  18857 non-null  int64  
 1   rodata      18857 non-null  int64  
 2   clube       18857 non-null  object 
 3   cartao      18857 non-null  object 
 4   atleta      18851 non-null  object 
 5   num_camisa  18471 non-null  float64
 6   posicao     17659 non-null  object 
 7   minuto      18857 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 1.2+ MB
None
Dataframe de stats info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16810 entries, 0 to 16809
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   partida_id       16810 non-null  int64 
 1   rodata           16810 non-null  int64 
 2   clube            16810 non-null  object
 3   chutes           16810 non

In [42]:
### Função para unir as tabelas com relação 1:2
def unir_tabelas_um_para_dois_chaves_diferentes(tabela1, tabela2, chave_tabela1, chave_tabela2, colunas_tabela2):
    """
    Une duas tabelas (um para dois) transformando as duas linhas correspondentes na tabela2 em duas colunas distintas,
    quando a chave de ligação entre as tabelas tem nomes diferentes.

    Parâmetros:
    tabela1 (pd.DataFrame): A tabela do lado 'um' da relação.
    tabela2 (pd.DataFrame): A tabela do lado 'dois' da relação.
    chave_tabela1 (str): O nome da coluna chave na tabela1.
    chave_tabela2 (str): O nome da coluna chave na tabela2.
    colunas_tabela2 (list): Lista de colunas da tabela2 que devem ser transformadas em colunas únicas na nova tabela.

    Retorno:
    pd.DataFrame: A tabela resultante da união."""
    
    
    # Primeiro, adicionamos um identificador único para distinguir as duas entradas (pode ser uma numeração)
    tabela2['linha'] = tabela2.groupby(chave_tabela2).cumcount() + 1
    
    # Fazemos o pivot para transformar as duas linhas em duas colunas
    tabela_pivot = tabela2.pivot(index=chave_tabela2, columns='linha', values=colunas_tabela2)
    
    # Renomeamos as colunas para que fiquem claras
    tabela_pivot.columns = ['{}_{}'.format(col, val) for col, val in tabela_pivot.columns]
    
    # Juntamos a tabela pivotada com a tabela original (tabela1)
    resultado_final = pd.merge(tabela1, tabela_pivot, left_on=chave_tabela1, right_on=chave_tabela2, how='left')
    
    return resultado_final

df_merged = unir_tabelas_um_para_dois_chaves_diferentes(tabela1=df_full, tabela2=df_stats_full, chave_tabela1='ID', chave_tabela2='partida_id',colunas_tabela2=['clube','chutes','chutes_no_alvo','posse_de_bola','passes','precisao_passes','faltas','cartao_amarelo','cartao_vermelho','impedimentos','escanteios'])
print(df_merged.head())
print(df_merged.info())
print(df_merged.describe())

   ID  rodata        data   hora       mandante    visitante  \
0   1       1  29/03/2003  16:00        Guarani        Vasco   
1   2       1  29/03/2003  16:00   Athletico-PR       Gremio   
2   3       1  30/03/2003  16:00       Flamengo     Coritiba   
3   4       1  30/03/2003  16:00          Goias     Paysandu   
4   5       1  30/03/2003  16:00  Internacional  Ponte Preta   

  formacao_mandante formacao_visitante tecnico_mandante tecnico_visitante  \
0               NaN                NaN              NaN               NaN   
1               NaN                NaN              NaN               NaN   
2               NaN                NaN              NaN               NaN   
3               NaN                NaN              NaN               NaN   
4               NaN                NaN              NaN               NaN   

       vencedor              arena  mandante_Placar  visitante_Placar  \
0       Guarani     Brinco de Ouro                4                 2   
1  Ath